In [1]:
import datatable as dt
import pandas as pd
import warnings
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
warnings.filterwarnings('ignore')

DATA_PATH = '../riiid-test-answer-prediction'
data = dt.fread(f"{DATA_PATH}/train.csv")
data = data.to_pandas()
data = data.loc[data.content_type_id==False,:]

In [2]:
numeric_features = ['prior_question_elapsed_time']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_features = ['user_id', 'content_id']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data[numeric_features+categorical_features], data['answered_correctly'], test_size=0.3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(roc_auc_score(y_test, y_pred))

0.6320779294487137
